In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import unicodedata
from lxml.html import parse
from urllib.request import urlopen
import re
import json
from util import get

In [2]:
def get_introduction(soup):
    
    if soup.select('p')[0].get_text() == '抱歉，您所访问的页面不存在...':
        return 0
    
    text = ''
    for i in soup.select("div.lemma-summary"):
        for c in i:
            if c != '\n':
                for t in [t for t in c.children if t.name != 'sup']:  #去掉【1】这种类型的标签
                    text += t.string
                
    return text

In [3]:
def get_basic_info(soup):
    basic_info = {}
    try:
        for i in range(len(soup.select('div.basic-info')[0].select('dt'))) :
            basic_info[soup.select('div.basic-info')[0].select('dt')[i].get_text()
                      ] = soup.select('div.basic-info')[0].select('dd')[i].get_text()[1:-1]
    except IndexError:
        return np.nan
            
    return basic_info

In [4]:
def get_name(soup):
    return soup.select('div > dl > dd > h1')[0].string

In [5]:
def parse( url):
    if get(url) is None:
        return
    soup = BeautifulSoup(get(url).content.decode(), 'lxml')
    dic = {}
    dic['url'] = url
    dic['name'] = get_name(soup)
    dic['description'] = get_introduction(soup)
    dic['metadata'] = get_basic_info( soup)
    return dic

In [14]:
if __name__ == '__main__':

    url_list = ['https://baike.baidu.com/item/京东','https://baike.baidu.com/item/蚂蚁金服']
    
    for u in url_list:
        
         with open(u.split('/')[-1]+'.json', 'w') as json_file:
            json.dump(parse(u), json_file, indent=4)